In [1]:
import pandas as pd
import numpy as np

In [2]:
robbins_craters = pd.read_csv("./lunar_crater_database_robbins_2018.csv")
yang_aged_craters = pd.read_csv("./Aged_Lunar_Crater_Database_DeepCraters_2020.csv")

The Yang 2020 database contains the ages for all craters larger than 10km in diameter. We need to process the Robbins 2018 database to find all craters less than 10km.

In [3]:
lt_10_inds = robbins_craters["DIAM_CIRC_IMG"] < 10
robbins_craters = robbins_craters[lt_10_inds]

For the sake of making modular arithmatic easier, we will add 360 to all negative longitudes in both databases.

In [4]:
def correct_lon_column(df, lon_name):
    """
    Add 360 to all negative longitude values in dataframe.

    Has the side effect of modifying the passed in dataframe.

    df: Pandas dataframe
    name: Name of column in the dataframe
    """
    def bool_to_deg(x):
        if x:
            return 360
        return 0
    neg_lon = df[lon_name] < 0
    lon_correct_factor = [bool_to_deg(i) for i in neg_lon]
    df[lon_name] = df[lon_name] + lon_correct_factor

correct_lon_column(robbins_craters, "LON_CIRC_IMG")
correct_lon_column(yang_aged_craters, "Lon")

We will now look at every crater smaller than 10km in Robbins 2018, and see if it lies within Yang 2020.

To check if a child crater lies within a parent crater, we first calculate the distance between the centerpoints of each crater. Then, we check if that distance is smaller the radius of the parent crater. We used this method to create a new database.

- `yang_id`: ID of crater in the Yang 2020 database
- `robbins_ids`: Set of IDs of craters in Robbins 2018 database that lie within a crater in the Yang database.

In [6]:
craters_in_craters = pd.read_csv("./robbins_in_yang.csv")

In [16]:
example_parent_crater = craters_in_craters["yang_id"][0]
example_child_craters = craters_in_craters["robbins_ids"][0]

print(f"ID of parent crater in Yang 2020: {example_parent_crater}")
print(f"IDs of child craters in Robbins 2018 within the parent: {example_child_craters}")

In [17]:
example_child_craters

"{'01-1-002004', '01-1-002609', '01-1-002759', '01-1-001987', '01-1-002610', '01-1-001985', '01-1-002611', '01-1-001989', '01-1-002604', '01-1-002040', '01-1-002708', '01-1-002765', '01-1-002760', '01-1-002608', '01-1-002624', '01-1-002621', '01-1-002692', '01-1-001953', '01-1-003912', '01-1-002001', '01-1-002002', '01-1-002600', '01-1-002023', '01-1-002003', '01-1-001993', '01-1-002701', '01-1-002021', '01-1-002012', '01-1-002054', '01-1-000006', '01-1-002056', '01-1-002598', '01-1-002713', '01-1-002705', '01-1-002684', '01-1-002703', '01-1-002602', '01-1-002710', '01-1-002704', '01-1-002009', '01-1-002011', '01-1-002683', '01-1-002024', '01-1-002686', '01-1-001955', '01-1-002599', '01-1-002693', '01-1-003913', '01-1-002696', '01-1-002690', '01-1-003914', '01-1-002712', '01-1-002025', '01-1-002620', '01-1-002709', '01-1-002688', '01-1-002006', '01-1-002605', '01-1-002613', '01-1-001995', '01-1-002768', '01-2-027898', '01-1-002702', '01-1-002053', '01-1-002612', '01-1-002008', '01-1-00